In [52]:
from langchain.text_splitter import TextSplitter

class DocumentSplitter(TextSplitter):
    def split_text(self, text: str):
        # 예시로 CSV를 한 행씩 분리하는 방법
        rows = text.split("\n")
        return [{"text": row} for row in rows if row]

csv_data = "Name, Age, Address\nAlice, 30, NY\nBob, 25, LA"
splitter = DocumentSplitter(chunk_size=50, chunk_overlap=0)
chunks = splitter.split_text(csv_data)

print(chunks)


[{'text': 'Name, Age, Address'}, {'text': 'Alice, 30, NY'}, {'text': 'Bob, 25, LA'}]


In [213]:
import os
from langchain_community.document_loaders import TextLoader, JSONLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# OpenAI API 키 설정
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)  # 금융 추천에 적합한 톤과 안정성을 유지

In [214]:
loader = CSVLoader("./data/fss_data.csv")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [66]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
def get_purpose(question):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = """
        Q의 목적을 다음 중 선택하여 출력하시오.
        - 예금/적금 추천
        - 계산
            (확실하게 계산을 해야 하는 경우)
        - 기타
            (대부분의 일반적인 경우)
        오직 이 중에 하나를 선택해서 그 것만을 출력합니다.
        Q.
    """
    result = llm.invoke(prompt + question)
    return result.content


def get_price(question):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = """
        Q에서 총 투자금을 추출합니다. 투자금만 -원으로 출력합니다.
        Q.
    """
    result = llm.invoke(prompt + question)
    return result.content

In [215]:
system_message_dict = {
    "예금/적금 추천": """
    당신은 사실만을 말하는 친절한 금융 전문가입니다. **이전 대화**를 적극 참고하여, 주어진 {context}를 바탕으로 사용자에게 적절한 금융 상품을 추천하는 역할을 합니다.
    - {context}에서 [fin_prdt_cd]이 같은 optionList를 찾아 다양한 정보를 비교하세요.
    - 하나의 상품으로 목표 달성이 어려우면 **여러 개**의 다양한 상품을 추천하여 목표를 달성하도록 합니다.
    - {question}과 [history]에서 아래 정보가 모두 주어지지 않으면 필요한 정보를 요구합니다.
        - 예금/적금
        - 적금 종류
        - 저축 금액
        - 저축 기간
        * 추천할 수 있는 정도의 정보가 모이면 추천을 시작합니다.
    
    - {context}에서 추출한 **실제 회사명**과 **상품명** 만을 사용합니다. 임의로 예시를 만들면 절대 안됩니다.
    - 금리와 같은 중요한 정보를 명확하게 제공합니다.
    - 특이사항을 제공합니다.
    - 제품을 요약합니다.
    - 
    사용자의 상황을 분석하고,
    예금과 적금을 적절하게 사용하여 기간 내에 금액을 만드는 전략 시나리오를 만듭니다.
    이후 해당 시나리오를 수행할 수 있는 금융 상품을 추천합니다.
    """,
    "계산": """
    당신은 유능한 금융 전문가입니다. 사용자의 {question}이 적금인지 예금인지 판단하고 기간과 금액, 이자율에 따라 얻을 수 있는 이득을 계산합니다.
    **이전 대화**를 적극 참고하여
    예금은 예치금과 기간이 주어질 것이고, 적금은 월 저축금과 기간이 주어질 것입니다.
    - 각 이득을 계산하는 과정을 수식과 함께 자세히 서술
    - 이자율 대신 상품명이 있다면 {context}에서 찾아 계산
    """,
    "기타": "당신은 친절한 금융 전문가입니다. 가능하면 {question}에 대해 {context}를 기반으로 대답합니다. 굳이 억지로 말을 만들지 않습니다."
}

In [ ]:
"계산"할 때는
    당신은 정밀한 금융 전문가입니다. 사용자가 요청한 {question}에 따라 예금과 적금 중 어떤 상품인지 판단하여, 기간, 금액, 그리고 이자율을 토대로 사용자에게 얻을 수 있는 이득을 계산합니다.
    
    - 사용자의 {chat_history}를 참고하여 예금과 적금에 대한 세부 정보를 확보합니다.
    - 사용자가 특정 상품을 언급했다면 {context}에서 관련 정보를 확인하여 이자율을 찾아 계산에 반영합니다.
    - 이득을 계산하는 과정과 계산 수식을 상세히 서술하며, 명확하고 직관적으로 설명합니다.

"일반적인 경우"
    "당신은 고객의 재정적 질문에 대해 신뢰할 수 있는 정보를 제공하는 금융 전문가입니다. 가능하다면 {question}에 대해 {context}를 기반으로 답변하고, 필요한 추가 정보를 요청하거나 명확한 설명을 제공하세요."

In [ ]:
system_message = '''
* {chat_history} 기록을 적극 참고하여, 필요할 경우 추가 정보를 요구하거나 관련된 질문을 유도하세요.

당신은 고객의 재무 목표 달성을 돕는 친절하고 정보에 정통한 금융 전문가입니다. {chat_history}를 참조하여, 주어진 {context}와 관련된 정보를 바탕으로 사용자에게 맞춤형 금융 상품을 추천합니다.
{question}을 분석하여 답을 해주세요.
1. {context}에서 [fin_prdt_cd]이 같은 optionList를 확인하여 최적의 상품을 비교하고 선정합니다.
2. {question}과 {chat_history}에 추천을 시작하기에 충분한 정보가 확보되면 상품 추천을 진행합니다. 은행명과 상품명을 명시합니다.
    - 상품 추천 시 {context}의 내용을 그대로 사용하며, 임의로 수정하지 마세요.
    - 금리와 같은 필수 정보를 명확하게 제공하며, 추천하는 이유와 특이사항을 설명합니다.
    - 제안된 각 상품의 요약과 함께 사용자의 상황에 맞춘 금액 달성 전략 시나리오를 제시할 수 있습니다.
    - 단일 상품으로 목표 달성이 어려울 경우, {context}에서 여러 개의 상품을 조합할 수 있습니다.
'''


In [230]:
loader_bank = CSVLoader("./data/bank_eval.csv", encoding='cp949')
documents_bank = loader_bank.load()


def get_price(question):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = """
        Q에서 총 투자금을 추출한 합산이 4500만원을 넘으면 "주의"를 출력합니다.
        아닌 경우에는 모두 "보통"으로 출력합니다.
        Q.
    """
    result = llm.invoke(prompt + question)
    return result.content


embeddings_bank = OpenAIEmbeddings()
vectorstore_bank = Chroma.from_documents(documents=documents_bank, embedding=embeddings_bank)
def get_eval(question):
    system_message = """
        1. {question}에서 등장한 은행을 추출합니다.
        2. {context}에거 추출한 은행의 정보를 찾습니다.
        평가 방법:
        재무상태표, 손익계산서, 자본적정성평가, 수익성평가, 여신건전성 평가 등을 종합하여 은행 건전성 평가에 활용

        국내 일반은행 평균
        BIS : 16.64
        고정이하여신 : 0.52
        ROA : 0.58
        NIM : 1.60

        국내 저축은행 평균
        BIS : 15.00
        고정이하여신 : 11.53
        ROA : 0.00  **(-)인 기관이 있음. 

        BIS(자기자본비율) 최소준수 비율 8%, 높을수록 좋다.
        NPL(고정이하여신비율)은 낮을수록 좋다.
        ROA(총자산이익률)은 높을수록 좋다.
        NIM(순이자마진)은 낮을수록 좋다.

        금융 평가 전문가인 당신은 추출한 은행의 정보를 평가하여 제공합니다.
    """
    qa_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=system_message
    )

    # Set up RetrievalQA with the correct combination of LLM and retriever
    chain = RetrievalQA.from_chain_type(
        llm=llm,  # Pass the llm directly
        retriever=vectorstore_bank.as_retriever(),
        memory=memory,
        chain_type_kwargs={"prompt": qa_prompt}
    )

    result = chain({'question': question, 'query': question})
    return result['result']

In [233]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from pprint import pprint

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

qa_prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template=system_message
)

chain = RetrievalQA.from_chain_type(
    llm=llm,  # Pass the llm directly
    retriever=vectorstore.as_retriever(),
    memory=memory,
    chain_type_kwargs={"prompt": qa_prompt, "memory": memory}
)


def qna(question):
    # purpose = get_purpose(question).replace('-', '').strip()

    # print('목적:', purpose)

    result = chain({'question': question, 'query': question, 'chat_history': memory.chat_memory.messages})['result']

    if "주의" in get_price(result):
        result += '\n' + get_eval(result)
    return result

while True:
    question = input('질문')
    if question == '끝':
        break
    result = qna(question)
    pprint(f"질문: {question}")
    pprint(f"답변: {result}")
    pprint("-" * 50)


'질문: 3년안에 5000만원 모으기'
('답변: 5000만원을 3년 안에 모으기 위한 전략을 세우기 위해, 국민은행의 자산 추세와 금융 상품을 고려해 보겠습니다.\n'
 '\n'
 '### 1. 자산 추세 분석\n'
 '국민은행의 자산은 다음과 같이 증가하고 있습니다:\n'
 '- 2022년 12월: 500,343,475 백만원\n'
 '- 2023년 6월: 506,394,830 백만원\n'
 '- 2023년 12월: 512,372,841 백만원\n'
 '- 2024년 6월: 533,335,407 백만원\n'
 '\n'
 '자산이 지속적으로 증가하고 있으며, 이는 안정적인 금융 기관임을 보여줍니다.\n'
 '\n'
 '### 2. 목표 설정\n'
 '3년 안에 5000만원(50백만원)을 모으기 위해 매년 모아야 할 금액은 다음과 같습니다:\n'
 '- 연평균 목표: 5000만원 / 3년 = 약 1666.67만원\n'
 '\n'
 '### 3. 추천 금융 상품\n'
 '국민은행에서 제공하는 금융 상품을 고려할 때, 다음과 같은 상품을 추천합니다:\n'
 '\n'
 '#### 1. **정기예금**\n'
 '- **금리**: 약 2.5% ~ 3.0% (변동 가능)\n'
 '- **특징**: 안전하고 안정적인 수익을 제공합니다. 만기까지 돈을 묶어두는 방식입니다.\n'
 '- **추천 이유**: 원금 보장이 되며, 안정적인 수익을 추구하는 고객에게 적합합니다.\n'
 '\n'
 '#### 2. **상호금융상품**\n'
 '- **금리**: 약 3.5% ~ 4.0%\n'
 '- **특징**: 상대적으로 높은 금리를 제공하며, 일정 기간 동안 자금을 묶어두는 방식입니다.\n'
 '- **추천 이유**: 높은 금리를 통해 목표 금액 달성에 유리합니다.\n'
 '\n'
 '### 4. 금액 달성 전략\n'
 '- **정기예금 활용**: 매년 1666.67만원을 정기예금에 투자하여 만기 시 이자를 포함한 금액을 수령합니다.\n'
 '- **상호금융상품 활용**: 만약 정기예

KeyboardInterrupt: 

In [166]:
chat_history = memory.chat_memory.messages

In [167]:
chat_history

[HumanMessage(content='1000만원 모으기', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"기타": "당신은 고객의 재정적 질문에 대해 신뢰할 수 있는 정보를 제공하는 금융 전문가입니다. 가능하다면 1000만원 모으기에 대해 에 액면금액을 지급한다.\n\n미쳐 장기금리가 변동하게 된다.\n\n전히 50만 원이지만, 복리를 적용한 상품은 1,050만 원에 5% 이자가 적용되기 때문에 이자가 52만 5천원이 됩니다. 이러한 과정이 5년간 반복되어 최종적 로 얻게 되는 이 자는 단리의 경우 250만 원이지만 복리의 경우는 276만 2천원 가량이 됩니다.\n\n5천 원, 분기납 1 만5천 원, 반기납 3만 원)가 적용 ?세제 혜택: 1인당 납입금액 최대 5,000만 원까지 비과세 ?가입 시 유의사항: 가입대상의 소득수준을 초과한다고 인정되는 경우 가입 불가를 기반으로 답변하고, 필요한 추가 정보를 요청하거나 명확한 설명을 제공하세요."', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='1000만원 모으기', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"기타": "당신은 고객의 재정적 질문에 대해 신뢰할 수 있는 정보를 제공하는 금융 전문가입니다. 가능하다면 1000만원 모으기에 대해 에 액면금액을 지급한다.\n\n미쳐 장기금리가 변동하게 된다.\n\n전히 50만 원이지만, 복리를 적용한 상품은 1,050만 원에 5% 이자가 적용되기 때문에 이자가 52만 5천원이 됩니다. 이러한 과정이 5년간 반복되어 최종적 로 얻게 되는 이 자는 단리의 경우 250만 원이지만 복리의 경우는 276만 2천원 가량이 됩니다.\n\n5천 원, 분기납 1 만5천 원, 반기납 3만 원)가 적용 ?세제 혜택: 1인당 납입금액 최대 5,000만 원까지 비과세 

In [129]:
result['answer']

'이해했습니다. 사용자의 요청에 따라 적절한 정보를 제공하겠습니다. \n\n사용자가 언급한 word: 물가안정목표제와 word: 기초가격에 대한 구별 및 추정에 대한 정보가 필요합니다. 추가 정보를 제공해 주시면, 맞춤형 금융 상품 추천이나 계산을 진행할 수 있습니다. \n\n다음 정보를 알려주실 수 있나요?\n- 예금 또는 적금 여부\n- 적금의 유형\n- 저축 금액 또는 월별 저축 금액\n- 저축 기간\n\n이 정보를 바탕으로 최적의 상품을 추천해 드리겠습니다.'

In [ ]:
tmp = """S:500만원을 1년 안에 모으고 싶으시다면,
적절한 적금 상품은 대한저축은행의 장학적금자유식입니다.
해당 상품은 2.4의 높은 금리를 가지고 있으며, 만19세 미만의 청소년만 가입 가능하고,
창구에서만 가입할 수 있는 특징이 있어 사용자님에게 적절하다고 판단됩니다. 추가적으로 우리은행의 WON플러스예금을 추천드립니다"""

print(chain("S:" + tmp + "\nS에서 은행명과 예적금을 모두 추출\n은행명-예적금")['result'])

대한저축은행-장학적금자유식  
우리은행-WON플러스예금


In [13]:
print(chain("저는 청년입니다.")['result'])
print(chain("제가 누구죠?")['result'])

청년이시군요! 어떤 도움이 필요하신가요?
모르겠습니다.


In [41]:
print('은행명: 대한저축은행  \n예적금 형태: 장학적금자유식')

은행명: 대한저축은행  
예적금 형태: 장학적금자유식


In [ ]:
chain = optimize_langchain_process()

questions = [
    "청년에게 좋은 담보대출은 무엇인가요?",
    "저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요",
    "그럼 월 상환액이 어느정도 되나요?"
]

prompt_template = """
    당신은 친절한 금융 전문가입니다. 당신의 역할은 주어진 정보를 바탕으로 사용자에게 작절한 금융 상품을 추천하는 것입니다. {fin_prdt_cd}이 같은 optionList를 찾아 다양한 정보를 비교하세요
    - 하나의 상품으로 목표 달성이 어려우면 여러 개의 상품을 추천하여 목표를 달성하도록 합니다.
    - 다음 예시와 같이 다양한 상품을 추천하세요. {}는 내용이 들어갈 공간이며 내용을 채운 후에는 삭제합니다.
    Q: info: {월급}, {예치금}, {기간}, {월 적립금}
    저에게 적절한 {is_deopsit} 상품을 추천해주세요.
    A: {금액}과 {기간}에 따라 추천드릴 {is_deposit} 상품은 {kor_co_nm}의 {fin_prdt_nm}입니다. 해당 상품은 {intr_rate}의 높은 금리를 가지고 있으며, {fin_prdt_info-_summation}과 같은 특징을 가지고 있어 사용자님에게 적절하다고 판단됩니다.
    {intr_rate}의 이자율로 계산을 했을 때 {compute_benefit} 정도의 이득을 볼 수 있습니다.
    가입 전에 {etc_note}과 같은 특이사항이 있으니 확인하시기 바랍니다. 이 상품으로는 목표 달성이 어려울 수 있으므로 다른 상품도 추천해드리겠습니다. {continue recommend}
"""

for i, question in enumerate(questions, 1):
    result = chain({"question": question})
    print(f"\n질문 {i}: {question}")
    print(f"답변 {i}: {result['answer']}")
    print("-" * 50)



질문 1: 청년에게 좋은 담보대출은 무엇인가요?
답변 1: 청년에게 좋은 담보대출은 LTV 비율이 30%~70%인 대출 상품이 좋습니다. 또한 연 이자율이 0.8% ~ 1.5% 범위 내에 있는 대출 상품을 선택하는 것이 좋습니다. 이러한 조건을 충족하는 대출 상품을 찾아보시는 것이 좋습니다.
--------------------------------------------------

질문 2: 저는 월 500만원 정도를 벌고 있습니다. 주택이 4억원이고 2억 정도를 대출받고 싶을 때 이에 적절한 상품이 있나요?, 상환 기간은 30년으로 생각하고 있어요
답변 2: 주택 가격이 4억원이고 2억 정도를 대출 받고 싶을 때, 월 수입이 500만원인 경우에는 대출 한도인 LTV 30%~70%를 고려해야 합니다. 주어진 상황에서는 4억원 중 2억을 대출 받고자 하므로 LTV 50%에 해당합니다. 따라서 대출 한도 내에서 대출을 받을 수 있습니다. 

다만, 월 수입이 500만원이고 대출 상환액을 감당할 수 있는지에 대한 판단은 개인의 재정 상황과 부담 가능 여부에 따라 다를 수 있습니다. 대출 상품의 이자율과 상환 기간 등을 고려하여 월 상환액을 계산하고, 개인의 재정 상황을 ganz히 고려하여 결정하는 것이 중요합니다.
--------------------------------------------------

질문 3: 그럼 월 상환액이 어느정도 되나요?
답변 3: 주어진 컨텍스트에 따르면 월 상환액은 대출 원금에 대한 이자와 원금 상환액으로 구성됩니다. 이자율은 대출금리와 추가 이자율인 3%를 합산하여 계산됩니다. 대출 원금은 LTV 비율에 따라 30%에서 70% 사이로 결정되며, 대출 기간은 3년입니다. 따라서 월 상환액은 대출 원금과 이자를 합산하여 계산됩니다. 상환 시작일은 2024년 10월 21일이며, 상환 종료일은 지정되어 있지 않습니다. 최종 제출일은 2024년 10월 18일 11시입니다. 월 상환액을 정확히 계산하려면 대출 원금과 이자율을 고려하여 계

In [55]:
loader_fin = CSVLoader("./data/경제금융용어.csv")
documents_fin = loader_fin.load()

embeddings_fin = OpenAIEmbeddings()
vectorstore_fin = Chroma.from_documents(documents=documents_fin, embedding=embeddings_fin)

RuntimeError: Error loading ./data/경제금융용어.csv

In [50]:
def get_prdt(question):
    parser = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = parser.invoke("Q:" + question + "\nQ에서 은행명과 예적금을 모두 추출\n은행명-예적금").content
    response = response.split(',')
    result = []
    for row in response:
        bank, prdt = row.split('-')
        result.append([{'bank':bank}, {'prdt':prdt}])
    return result

def get_fin(question):
    system_message = """
        {question}에서 등장한 용어 중 {context}에 없는 단어를 삭제합니다.
        format: 용어1,용어2,용어2
    """
    qa_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=system_message
    )

    # Set up RetrievalQA with the correct combination of LLM and retriever
    chain = RetrievalQA.from_chain_type(
        llm=llm,  # Pass the llm directly
        retriever=vectorstore_fin.as_retriever(),
        memory=memory,
        chain_type_kwargs={"prompt": qa_prompt}
    )

    result = chain({'query': question})
    return result['result'].split(',')

In [26]:
get_prdt('부산은행 적금, K뱅크 예금')

[[{'bank': '부산은행'}, {'prdt': '적금'}], [{'bank': ' K뱅크'}, {'prdt': '예금'}]]

In [51]:
get_fin("예금은 좋고, 적금은 들기 힘듧니다.")

['예·적금 수익률 높이기', ' 예금', ' 적금']

In [10]:
tmp = """S:500만원을 1년 안에 모으고 싶으시다면,
적절한 적금 상품은 대한저축은행의 장학적금자유식입니다.
해당 상품은 2.4의 높은 금리를 가지고 있으며, 만19세 미만의 청소년만 가입 가능하고,
창구에서만 가입할 수 있는 특징이 있어 사용자님에게 적절하다고 판단됩니다. 추가적으로 우리은행의 WON플러스예금을 추천드립니다"""

result = get_prdt(tmp)

In [11]:
result

'대한저축은행 - 장학적금자유식  \n우리은행 - WON플러스예금'

In [191]:
loader_bank = CSVLoader("./data/bank_eval.csv", encoding='cp949')
documents_bank = loader_bank.load()

embeddings_bank = OpenAIEmbeddings()
vectorstore_bank = Chroma.from_documents(documents=documents_bank, embedding=embeddings_bank)

In [192]:
def get_eval(question):
    system_message = """
        1. {question}에서 등장한 은행을 추출합니다.
        2. {context}에거 추출한 은행의 정보를 찾습니다.
        평가 방법:
        재무상태표, 손익계산서, 자본적정성평가, 수익성평가, 여신건전성 평가 등을 종합하여 은행 건전성 평가에 활용

        국내 일반은행 평균
        BIS : 16.64
        고정이하여신 : 0.52
        ROA : 0.58
        NIM : 1.60

        국내 저축은행 평균
        BIS : 15.00
        고정이하여신 : 11.53
        ROA : 0.00  **(-)인 기관이 있음. 

        BIS(자기자본비율) 최소준수 비율 8%, 높을수록 좋다.
        NPL(고정이하여신비율)은 낮을수록 좋다.
        ROA(총자산이익률)은 높을수록 좋다.
        NIM(순이자마진)은 낮을수록 좋다.

        금융 평가 전문가인 당신은 추출한 은행의 정보를 평가하여 제공합니다.
    """
    qa_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=system_message
    )

    # Set up RetrievalQA with the correct combination of LLM and retriever
    chain = RetrievalQA.from_chain_type(
        llm=llm,  # Pass the llm directly
        retriever=vectorstore_bank.as_retriever(),
        memory=memory,
        chain_type_kwargs={"prompt": qa_prompt}
    )

    result = chain({'question': question, 'query': question})
    return result['result'].split(',')

In [193]:
result = get_eval("CK저축은행")

In [194]:
pprint(result)

['CK저축은행의 재무정보를 바탕으로 은행의 건전성을 평가해 보겠습니다. 제공된 데이터는 자산과 부채에 대한 정보입니다.\n'
 '\n'
 '### 1. 자산 및 부채 현황\n'
 '- **자산 (백만원)**\n'
 '  - 2022년 12월: 262',
 '939\n  - 2023년 06월: 330',
 '530\n  - 2023년 12월: 421',
 '549\n  - 2024년 06월: 473',
 '186\n\n- **부채 (백만원)**\n  - 2022년 12월: 232',
 '467\n  - 2023년 06월: 294',
 '159\n  - 2023년 12월: 379',
 '468\n  - 2024년 06월: 427',
 '853\n'
 '\n'
 '### 2. 자본 및 자기자본 비율(BIS) 계산\n'
 '자본은 자산에서 부채를 뺀 값으로 계산할 수 있습니다.\n'
 '\n'
 '- **자본 (백만원)**\n'
 '  - 2022년 12월: 262',
 '939 - 232',
 '467 = 30',
 '472\n  - 2023년 06월: 330',
 '530 - 294',
 '159 = 36',
 '371\n  - 2023년 12월: 421',
 '549 - 379',
 '468 = 42',
 '081\n  - 2024년 06월: 473',
 '186 - 427',
 '853 = 45',
 '333\n\n- **BIS 계산**\n  - BIS = (자본 / 자산) * 100\n  - 2022년 12월: (30',
 '472 / 262',
 '939) * 100 ≈ 11.58%\n  - 2023년 06월: (36',
 '371 / 330',
 '530) * 100 ≈ 11.01%\n  - 2023년 12월: (42',
 '081 / 421',
 '549) * 100 ≈ 10.00%\n  - 2024년 06월: (45',
 '333 / 473',
 '186) * 100 ≈ 9.57%\n\n### 3. 고정이하여신 비율(NPL)\n고정이하여신 비율에 대